# Simularium Conversion Tutorial : Custom Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from any source. It requires the following data:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **times** : *np.ndarray (shape = \[timesteps\])*
    * A numpy ndarray containing the elapsed simulated time at each timestep
* **n_agents** : *np.ndarray (shape = \[timesteps\])*
    * A numpy ndarray containing the number of agents that exist at each timestep
* **viz_types** : *np.ndarray (shape = \[timesteps, agents\])*
    * A numpy ndarray containing the viz type for each agent at each timestep. Current options:
        * 1000 : default,
        * 1001 : fiber (which will require subpoints)
* **unique_ids** : *np.ndarray (shape = \[timesteps, agents\])*
    * A numpy ndarray containing the unique ID for each agent at each timestep
* **types** : *List\[List\[str\]\] (list of shape \[timesteps, agents\])*
    * A list containing timesteps, for each a list of the string name for the type of each agent
* **positions** : *np.ndarray (shape = \[timesteps, agents, 3\])*
    * A numpy ndarray containing the XYZ position for each agent at each timestep
* **radii** : *np.ndarray (shape = \[timesteps, agents\])*
    * A numpy ndarray containing the radius for each agent at each timestep
    * optional only if all agents are fibers
    
Optionally, you can also include:
* **n_subpoints** : *np.ndarray (shape = \[timesteps, agents\]) (optional)*
    * A numpy ndarray containing the number of subpoints belonging to each agent at each timestep. Required if subpoints are provided
* **subpoints** : *np.ndarray (shape = \[timesteps, agents, subpoints, 3\])*
    * A numpy ndarray containing a list of subpoint position data for each agent at each timestep. These values are currently only used for fiber agents.
* **plots** : *Dict\[str, Any\]*
    * An object containing plot data already in Simularium format

### Generate example data

To demonstrate using the custom converter, we'll first generate some random example data.

In [2]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10
agents_are_fibers = False
points_per_fiber = 4

example_data = {
    "box_size" : np.array([box_size, box_size, box_size]),
    "times" : timestep * np.array(list(range(total_steps))),
    "n_agents" : np.array(total_steps * [n_agents]),
    "unique_ids" : np.array(total_steps * [list(range(n_agents))]),
    "types" : []
}
for t in range(total_steps):
    example_data["types"].append([choice(ascii_uppercase) for i in range(n_agents)])

### Sphere visualization

If the agents are particles to be visualized as spheres, their centroid positions can be added like this:

In [3]:
if not agents_are_fibers:
    example_data["positions"] = np.random.uniform(size=(total_steps, n_agents, 3)) * box_size - box_size * 0.5
    example_data["viz_types"] = np.array(total_steps * [n_agents * [1000.0]]) # default viz type = 1000
    example_data["radii"] = (max_radius - min_radius) * np.random.uniform(size=(total_steps, n_agents)) + min_radius

### Line visualization

If the agents are fibers to be visualized as lines, the positions of points along each fiber can be added using the optional subpoints:

In [4]:
if agents_are_fibers:
    example_data["positions"] = np.zeros(shape=(total_steps, n_agents, 3))
    example_data["n_subpoints"] = points_per_fiber * np.ones(shape=(total_steps, n_agents))
    example_data["subpoints"] = box_size * np.random.uniform(
        size=(total_steps, n_agents, points_per_fiber, 3)) - box_size * 0.5
    example_data["viz_types"] = np.array(total_steps * [n_agents * [1001.0]]) # fiber viz type = 1001
    # no need to set radii since all agents are fibers

Inspect the resulting data:

In [5]:
print("box size = {}".format(example_data["box_size"]))
print("times = {}".format(example_data["times"]))
print("number of agents = {}".format(example_data["n_agents"]))
print("viz types = {}".format(example_data["viz_types"]))
print("unique IDs = {}".format(example_data["unique_ids"]))
print("types = {}".format(example_data["types"]))
print("positions = {}".format(example_data["positions"]))
print("radii = {}".format(example_data["radii"] if "radii" in example_data else "None"))
if agents_are_fibers:
    print("n_subpoints = {}".format(example_data["n_subpoints"]))
    print("subpoints = {}".format(example_data["subpoints"]))

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [6]:
converter = Converter(example_data)
converter.write_JSON("/Users/blairl/Desktop/example")

***
## Add plots

You can optionally include metrics data, which will be graphed in plots alongside your spatial data in the Simularium viewer. For now, Simularium supports scatterplots and histograms.

### Scatterplots

Scatterplots require the following data:
* **title** : *str*
    * A string display title for the plot
* **xaxis_title** : *str*
    * A string label (with units) for the x-axis
* **yaxis_title** : *str*
    * A string label (with units) for the y-axis
* **xtrace** : *np.ndarray (shape = [x values])*
    * A numpy ndarray of values for x, the independent variable
* **ytraces** : *Dict[str, np.ndarray (shape = [x values])]*
    * A dictionary with y-trace display names as keys, each mapped to a numpy ndarray of values for y, the dependent variable
* **render_mode** : *str (optional)*
    * A string specifying how to draw the datapoints. Options:
        * 'markers' : draw as points
        * 'lines' : connect points with line
    * Default: 'markers'

Here's some random example scatterplot data:

In [7]:
scatter_plot = {
    "title": "Test Scatterplot 1",
    "xaxis_title": "time (ns)",
    "yaxis_title": "concentration (uM)",
    "xtrace": np.array(list(range(10))),
    "ytraces": {
        "agent1": 100 * np.random.uniform(size=(10)),
        "agent2": 100 * np.random.uniform(size=(10)),
        "agent3": 100 * np.random.uniform(size=(10)),
    },
    "render_mode": "lines"
}

In [8]:
print("title = {}".format(scatter_plot["title"]))
print("xaxis_title = {}".format(scatter_plot["xaxis_title"]))
print("yaxis_title = {}".format(scatter_plot["yaxis_title"]))
print("xtrace = {}".format(scatter_plot["xtrace"]))
print("ytraces:")
print("  agent1 = {}".format(scatter_plot["ytraces"]["agent1"]))
print("  agent2 = {}".format(scatter_plot["ytraces"]["agent2"]))
print("  agent2 = {}".format(scatter_plot["ytraces"]["agent3"]))
print("render_mode = {}".format(scatter_plot["render_mode"]))

title = Test Scatterplot 1
xaxis_title = time (ns)
yaxis_title = concentration (uM)
xtrace = [0 1 2 3 4 5 6 7 8 9]
ytraces:
  agent1 = [85.81810117 63.93254575 66.32377308 78.92439753 14.15277033 95.34283801
 44.47598818 47.67725994 42.52787767 44.17199114]
  agent2 = [18.04285137 54.62098886 28.57397349 91.8222     61.20877391 13.83121966
 64.4702447  58.14381492 70.58668005 99.38155863]
  agent2 = [25.22929223 85.81606452 71.55704031  0.73093903 49.32314128 86.66234291
 93.81519382 10.78393371 22.60652272 19.77288185]
render_mode = lines


To add it to the converter:

In [9]:
converter.add_plot(scatter_plot, "scatter")

### Histograms

Histograms require the following data:
* **title** : *str*
    * A string display title for the plot
* ***xaxis_title*** : *str*
    * A string label (with units) for the x-axis
* **traces** : *Dict[str, np.ndarray (shape = [values])]*
    * A dictionary with trace display names as keys, each mapped to a numpy ndarray of values

Here's some random example histogram data:

In [10]:
histogram = {
    "title": "Test Histogram 1",
    "xaxis_title": "angle (degrees)",
    "traces": {
        "crosslinked monomers": 2 * np.random.uniform(size=(15)),
        "bent monomers": 10 * np.random.uniform(size=(10)),
    }
}

In [11]:
print("title = {}".format(histogram["title"]))
print("xaxis_title = {}".format(histogram["xaxis_title"]))
print("traces:")
print("  crosslinked monomers = {}".format(histogram["traces"]["crosslinked monomers"]))
print("  bent monomers = {}".format(histogram["traces"]["bent monomers"]))

title = Test Histogram 1
xaxis_title = angle (degrees)
traces:
  crosslinked monomers = [1.68973839 0.0519339  1.5287687  1.92569546 1.20478972 0.15735779
 0.04433263 0.31003189 0.28609787 1.41378324 1.71610002 1.02545611
 0.82127704 1.27196588 1.42048629]
  bent monomers = [9.73713118 6.15160034 2.49780667 7.56833878 4.6585807  2.19882019
 0.40965719 6.51399356 8.88679448 4.24076873]


To add it to the converter:

In [12]:
converter.add_plot(histogram, "histogram")

### Save the data with added plots

Once you've added your plot data, write the updated data to file:

In [13]:
converter.write_JSON("/Users/blairl/Desktop/example")

## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*